In [1]:
print("OK")

OK


In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [3]:
# Read the PDF files from the folder

file_dir = "../dataset"
loader = PyPDFDirectoryLoader(file_dir)
docs = loader.load()

In [4]:
len(docs)

27

In [5]:
docs[11].page_content

'BUILD RESILIENT INFRASTRUCTURE, \nPROMOTE INCLUSIVE AND SUSTAINABLE \nINDUSTRIALIZATION AND FOSTER \nINNOVATION \nTechnological progress helps us address big global challenges such as \ncreating jobs and becoming more energy efficient. For example, the \nworld is becoming ever more interconnected and prosperous thanks to \nthe internet. The more connected we are, the more we can all benefit \nfrom the wisdom and contributions of people everywhere on earth.  And \nyet four billion people have no way of getting online, the vast majority \nof them in developing countries. The more we invest in innovation \nand infrastructure, the better off we’ll all be. Bridging the digital divide, \npromoting sustainable industries, and investing in scientific research and \ninnovation are all important ways to facilitate sustainable development.'

In [6]:
# Extract the text from the PDF
pdf_str = ""

for page in docs:
    pdf_str += page.page_content

In [7]:
pdf_str[0:25]

'IN THE YEAR 2015, LEADERS'

In [8]:
len(pdf_str)

19986

In [9]:
# Split the documents into chunks
model = "gpt-3.5-turbo"

text_splitter = TokenTextSplitter(
    model_name = model,
    chunk_size = 10000,
    chunk_overlap = 200
)
pdf_doc = text_splitter.split_text(pdf_str)

In [10]:
pdf_doc

['IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15

In [11]:
len(pdf_doc)

1

In [12]:
type(pdf_doc[0])

str

In [13]:
# Convert string to langchain documents
from langchain.docstore.document import Document

doc = [Document(page_content = t) for t in pdf_doc]

In [14]:
doc[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [15]:
type(doc[0])

langchain_core.documents.base.Document

In [16]:
# Split the documents into chunks
model = "gpt-3.5-turbo"

text_splitter = TokenTextSplitter(
    model_name = model,
    chunk_size = 1000,
    chunk_overlap = 100
)
pdf_docs = text_splitter.split_documents(doc)

In [17]:
pdf_docs[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [18]:
len(pdf_docs)

6

In [19]:
# Import ollama chat model
from langchain_community.chat_models import ChatOllama

In [20]:
# Define llm model
model_name = "llama3.2:1b"

model = ChatOllama(
    model = model_name,
    temperature = 0.3
)

C:\Users\hamza\AppData\Local\Temp\ipykernel_5940\3396867538.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [21]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [22]:
# Designing a ChatPrompt Template
from langchain_core.prompts import PromptTemplate

In [23]:
prompt_questions = PromptTemplate(
    template = prompt_template,
    input_variables = ["text"]
)

In [24]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.

------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [25]:
refine_prompt_questions = PromptTemplate(
    input_variables = ["existing_answer", "text"],
    template = refine_template
)

In [26]:
"""
    The refine documents chain constructs a response by looping over the input documents and iteratively updating 
    its answer. For each document, it passes all non-document inputs, the current document, and the latest 
    intermediate answer to an LLM chain to get a new answer
"""
from langchain.chains.summarize import load_summarize_chain

In [27]:
ques_chain = load_summarize_chain(
    llm = model,
    chain_type = "refine",
    verbose = True,
    question_prompt = prompt_questions,
    refine_prompt = refine_prompt_questions
)

In [28]:
ques = ques_chain.run(doc)

C:\Users\hamza\AppData\Local\Temp\ipykernel_5940\210017599.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_chain.run(doc)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders f

In [29]:
print(ques)

Here are some potential questions that could be asked to prepare coders or programmers for their tests on correlation:

**General Questions**

1. What is correlation, and how does it differ from causation?
2. Can you explain the difference between a correlation coefficient and a regression coefficient?
3. How do you interpret the results of a correlation analysis?

**Data Analysis Questions**

1. Given a dataset with two variables (e.g., temperature and humidity), what type of relationship can be expected to exist between them?
2. If you have a dataset with multiple variables, how would you identify the most relevant relationships between them?
3. How do you handle missing values in a correlation analysis?

**Data Visualization Questions**

1. What is the purpose of creating scatter plots or bar charts in a correlation analysis?
2. Can you explain the difference between a histogram and a box plot in terms of their use in correlation analysis?
3. How can you effectively communicate the 

In [30]:
embeds = OllamaEmbeddings(
    model = model_name
)

C:\Users\hamza\AppData\Local\Temp\ipykernel_5940\1706272247.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeds = OllamaEmbeddings(


In [31]:
db = FAISS.from_documents(pdf_docs, embeds)

In [32]:
foler_path = "./embeds_index"
db.save_local(folder_path=foler_path)

In [33]:
new_db = FAISS.load_local(folder_path=foler_path, embeddings=embeds, allow_dangerous_deserialization=True)

In [34]:
llm = ChatOllama(model = model_name, temperature = 0.1)

In [35]:
ques

"Here are some potential questions that could be asked to prepare coders or programmers for their tests on correlation:\n\n**General Questions**\n\n1. What is correlation, and how does it differ from causation?\n2. Can you explain the difference between a correlation coefficient and a regression coefficient?\n3. How do you interpret the results of a correlation analysis?\n\n**Data Analysis Questions**\n\n1. Given a dataset with two variables (e.g., temperature and humidity), what type of relationship can be expected to exist between them?\n2. If you have a dataset with multiple variables, how would you identify the most relevant relationships between them?\n3. How do you handle missing values in a correlation analysis?\n\n**Data Visualization Questions**\n\n1. What is the purpose of creating scatter plots or bar charts in a correlation analysis?\n2. Can you explain the difference between a histogram and a box plot in terms of their use in correlation analysis?\n3. How can you effective

In [36]:
ques_list = ques.split("\n")

In [37]:
ques_list

['Here are some potential questions that could be asked to prepare coders or programmers for their tests on correlation:',
 '',
 '**General Questions**',
 '',
 '1. What is correlation, and how does it differ from causation?',
 '2. Can you explain the difference between a correlation coefficient and a regression coefficient?',
 '3. How do you interpret the results of a correlation analysis?',
 '',
 '**Data Analysis Questions**',
 '',
 '1. Given a dataset with two variables (e.g., temperature and humidity), what type of relationship can be expected to exist between them?',
 '2. If you have a dataset with multiple variables, how would you identify the most relevant relationships between them?',
 '3. How do you handle missing values in a correlation analysis?',
 '',
 '**Data Visualization Questions**',
 '',
 '1. What is the purpose of creating scatter plots or bar charts in a correlation analysis?',
 '2. Can you explain the difference between a histogram and a box plot in terms of their us

In [38]:
len(ques_list)

33

In [39]:
filtered_ques_list = [element for element in ques_list if element.endswith('?') or element.endswith('.')]

In [40]:
filtered_ques_list

['1. What is correlation, and how does it differ from causation?',
 '2. Can you explain the difference between a correlation coefficient and a regression coefficient?',
 '3. How do you interpret the results of a correlation analysis?',
 '1. Given a dataset with two variables (e.g., temperature and humidity), what type of relationship can be expected to exist between them?',
 '2. If you have a dataset with multiple variables, how would you identify the most relevant relationships between them?',
 '3. How do you handle missing values in a correlation analysis?',
 '1. What is the purpose of creating scatter plots or bar charts in a correlation analysis?',
 '2. Can you explain the difference between a histogram and a box plot in terms of their use in correlation analysis?',
 '3. How can you effectively communicate the results of a correlation analysis to non-technical stakeholders?',
 '1. A marketing manager wants to identify the relationship between advertising spending and sales. What ty

In [41]:
len(filtered_ques_list)

16

In [42]:
# Create the RetrievalQA chain for answering questions
from langchain.chains import RetrievalQA

In [43]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm, 
                                               chain_type="stuff", 
                                               retriever=db.as_retriever()
                                            )

In [45]:
# Answer each question and save to a file
for question in ques_list:
    print(f"Question: {question}")
    print("--------------------------------------------------")
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")
    break

Question: Here are some potential questions that could be asked to prepare coders or programmers for their tests on correlation:
--------------------------------------------------
Answer:  Here are some potential questions that could be asked to prepare coders or programmers for their tests on correlation:

**Theory and Concepts**

1. What is correlation, and how does it differ from causation?
2. Explain the concept of statistical significance and p-values.
3. Describe the different types of correlations (e.g., linear, non-linear, mutual).
4. How do you determine the strength and direction of a correlation coefficient (e.g., Pearson's r, Spearman's rho)?
5. What are some common pitfalls in interpreting correlation results (e.g., multicollinearity, outliers)?

**Data Analysis**

1. How would you load and preprocess data for analysis?
2. Describe your approach to handling missing values or outliers in a dataset.
3. Explain how to perform exploratory data analysis using visualization tool